# Calibrate dark images

Dark images, like any other images, need to be calibrated. Depending on the data
you have and the choices you have made in reducing your data, the steps to
reducing your images may include:

1. Subtracting overscan (only if you decide to subtract overscan from all
images).
2. Trim the image (if it has overscan, whether you are using the overscan or
not).
3. Subtract bias (if you need to scale the calibrated dark frames to a different
exposure time).

In [ ]:
from pathlib import Path

from astropy.nddata import CCDData
from ccdproc import ImageFileCollection
import ccdproc as ccdp

## Example 1: Overscan subtracted, bias not removed

### Take a look at what images you have

First we gather up some information about the raw images and the reduced images
up to this point. These examples have darks stored in a subdirectory of the
folder with the rest of the images, so we create an `ImageFileCollection` for
each.

In [ ]:
ex1_path_raw = Path('example-cryo-LFC')

ex1_images_raw = ImageFileCollection(ex1_path_raw)
ex1_darks_raw = ImageFileCollection(ex1_path_raw / 'darks')

ex1_path_reduced = Path('example1-reduced')
ex1_images_reduced = ImageFileCollection(ex1_path_reduced)

#### Raw images, everything except the darks

In [ ]:
ex1_images_raw.summary['file', 'imagetyp', 'exptime', 'filter']

#### Raw dark frames

In [ ]:
ex1_darks_raw.summary['file', 'imagetyp', 'exptime', 'filter']

### Decide which calibration steps to take

This example is, again, one of the chips of the LFC camera at Palomar. In
earlier notebooks we have seen that the chip has a [useful overscan region](01-08-Overscan.ipynb#case-1-cryogenically-cooled-large-format-camera-lfc-at-palomar), has little dark current except for some hot pixels, and sensor glow in
one corner of the chip.

Looking at the list of non-dark images (i.e., the flat and light images) shows
that for each exposure time in the non-dark images there is a set of dark
exposures that has a matching, or very close to matching, exposure time.

To be more explicit, there are flats with exposure times of 7.0 sec and 70.011
sec and darks with exposure time of 7.0 and 70.0 sec. The dark and flat exposure
times are close enough that there is no need to scale them.  The two images of
an object are each roughly 300 sec, matching the darks with exposure time 300
sec. The very small difference in exposure time, under 0.1 sec, does not need to
be compensated for.

Given this, we will:

1. Subtract overscan from each of the darks. The useful overscan region is XXX
(see LINK).
2. Trim the overscan out of the dark images.

We will *not* subtract bias from these images because we will *not* need to
rescale them to a different exposure time.

### Calibrate the individual dark frames

In [ ]:
for ccd, file_name in ex1_darks_raw.ccds(imagetyp='DARK',            # Just get the dark frames
                                         ccd_kwargs={'unit': 'adu'}, # CCDData requires a unit for the image if 
                                                                     # it is not in the header
                                         return_fname=True           # Provide the file name too.
                                        ):    
    # Subtract the overscan
    ccd = ccdp.subtract_overscan(ccd, overscan=ccd[:, 2055:], median=True)
    
    # Trim the overscan
    ccd = ccdp.trim_image(ccd[:, :2048])
    
    # Save the result
    ccd.write(ex1_path_reduced / file_name)

#### Reduced images (so far)

In [ ]:
ex1_images_reduced.refresh()
ex1_images_reduced.summary['file', 'imagetyp', 'exptime', 'filter', 'combined']

## Example 2: Overscan not subtracted, bias is removed

In [ ]:
ex2_path_raw = Path('example-thermo-electric')

ex2_images_raw = ImageFileCollection(ex2_path_raw)

ex2_path_reduced = Path('example2-reduced')
ex2_images_reduced = ImageFileCollection(ex2_path_reduced)

We begin by looking at what exposure times we have in this data.

In [ ]:
ex2_images_raw.summary['file', 'imagetyp', 'exposure'].show_in_notebook()

### Decide what steps to take next

In this case the only dark frames have exposure time 90 sec. Though that matches
the exposure time of the science images, the flat field images are much shorter
exposure time, ranging from 1 sec to 1.21 sec. This type of range of exposure is
typical when twilight flats are taken. Since these are a much different
exposure time than the darks, the dark frames will need to be scaled.

Recall that for this camera the overscan is not useful and should be
trimmed off.

Given this, we will:

1. Trim the overscan from each of the dark frames.
2. Subtract calibration bias from the dark frames so that we can scale the darks
to a different exposure time.

### Calibration the individual dark frames

First, we read the combined bias image created in the previous notebook. Though
we could do this based on the file name, using a systematic set of header
keywords to keep track of which images have been combined is less likely to lead
to errors.

In [ ]:
combined_bias = CCDData.read(ex2_images_reduced.files_filtered(imagetyp='bias', 
                                                               combined=True, 
                                                               include_path=True)[0])

In [ ]:
for ccd, file_name in ex2_images_raw.ccds(imagetyp='DARK',            # Just get the bias frames
                                          return_fname=True           # Provide the file name too.
                                         ):
        
    # Trim the overscan
    ccd = ccdp.trim_image(ccd[:, :4096])
    
    # Subtract bias
    ccd = ccdp.subtract_bias(ccd, combined_bias)
    # Save the result
    ccd.write(ex2_path_reduced / file_name)